For my solo exploration, I want to look at different countries over time and compare their total consumpiton of electric power in thousands of kW per year. I also want to show how much of that total consumption is produceed by means of nuclear power.   

In [26]:
import pandas as pd
import numpy as np 
import plotly.express as px

Read only data rows into the dataframes. 

In [22]:
total_electricity = pd.read_csv('../data/total_electricity.csv',nrows=100000)
nuclear_electricity = pd.read_csv('../data/nuclear_electricity.csv',nrows=915)

Drop the footnotes column for cleaner look. Check the data types of each of the columns. 

In [24]:
total_electricity = total_electricity.drop(columns=['Quantity Footnotes'])
nuclear_electricity = nuclear_electricity.drop(columns=['Quantity Footnotes'])
print('Total Electricity:\n',total_electricity.dtypes,'\n\nNuclear Electricity:\n',nuclear_electricity.dtypes)

Total Electricity:
 Country or Area             object
Commodity - Transaction     object
Year                         int64
Unit                        object
Quantity                   float64
dtype: object 

Nuclear Electricity:
 Country or Area             object
Commodity - Transaction     object
Year                         int64
Unit                        object
Quantity                   float64
dtype: object


Lets first look at the total electricity consumption for each country in the latest year of data, 2020. 

In [36]:
total_electricity_consumption = total_electricity[(total_electricity['Commodity - Transaction'] == 'Electricity - Final energy consumption')]

In [49]:
del test